In [ ]:
from chess import Board

board = Board()
board

In [ ]:
from chess import Move

Nf3 = Move.from_uci("g1f3")
board.push(Nf3)  # Make the move
board

In [ ]:
board.pop()
board

In [ ]:
print(board)

In [ ]:
print(board.is_stalemate())
print(board.is_insufficient_material())
print(board.outcome())

In [ ]:
print(board.can_claim_threefold_repetition())
print(board.halfmove_clock)
print(board.can_claim_fifty_moves())
print(board.can_claim_draw())

In [ ]:
print(board.is_fivefold_repetition())
print(board.is_seventyfive_moves())

In [ ]:
import chess

print(board.is_check())
print(board.is_attacked_by(chess.WHITE, chess.E8))

attackers = board.attackers(chess.WHITE, chess.F3)
print(attackers)

attackers

In [ ]:
board = chess.Board()
print(board.san(chess.Move(chess.E2, chess.E4)))
print(board.parse_san('Nf3'))
print(board.variation_san([chess.Move.from_uci(m) for m in ["e2e4", "e7e5", "g1f3"]]))

In [ ]:
print(board.fen())
print(board.shredder_fen())
board = chess.Board("8/8/8/2k5/4K3/8/8/8 w - - 4 45")
print(board.piece_at(chess.C5))

board.piece_at(chess.C5)

In [ ]:
board = chess.Board()
board.epd(bm=board.parse_uci("d2d4"))
ops = board.set_epd("1k1r4/pp1b1R2/3q2pp/4p3/2B5/4Q3/PPP2B2/2K5 b - - bm Qd1+; id \"BK.01\";")
ops == {'bm': [chess.Move.from_uci('d6d1')], 'id': 'BK.01'}

In [ ]:
import chess.polyglot

book = chess.polyglot.open_reader("data/polyglot/performance.bin")

board = chess.Board()
main_entry = book.find(board)
print(main_entry.move)
print(main_entry.weight)

book.close()

In [ ]:
import chess.pgn

with open("data/pgn/molinari-bordais-1979.pgn") as pgn:
    first_game = chess.pgn.read_game(pgn)

print(first_game.headers["White"])
print(first_game.headers["Black"])

first_game.mainline()  

first_game.headers["Result"]

In [ ]:
import chess.syzygy

tablebase = chess.syzygy.open_tablebase("data/syzygy/regular")

# Black to move is losing in 53 half moves (distance to zero) in this
# KNBvK endgame.
board = chess.Board("8/2K5/4B3/3N4/8/8/4k3/8 b - - 0 1")
tablebase.probe_dtz(board)

tablebase.close()

In [ ]:
import chess.engine

engine = chess.engine.SimpleEngine.popen_uci("stockfish")

board = chess.Board("1k1r4/pp1b1R2/3q2pp/4p3/2B5/4Q3/PPP2B2/2K5 b - - 0 1")
limit = chess.engine.Limit(time=2.0)
engine.play(board, limit)  

engine.quit()

In [ ]:
# from chess import Move
# import chess

# board = chess.Board()

# chess_history = "b3 Na6 Nh3 d5 a3 e6 Rg1 c6 Rh1 g6 c3 Bd7 Rg1 f5 Rh1 Kf7 Rg1 Ke8 Rh1 Kf7 Rg1 Ke8 Rh1 Kf7 Rg1 Ke8 Rh1 Kf7 Rg1 Ke8 Rh1 Kf7 Rg1 Ke8 Rh1 Kf7 Rg1 Ke8 Rh1 Kf7 Rg1 Ke8 Rh1 Kf7 Rg1 Ke8 Rh1 Kf7 Rg1 Ke8 Rh1 Kf7 Rg1 Ke8 Rh1 Kf7 Rg1 Ke8 Rh1 Kf7 Rg1 Ke8 Rh1 Kf7 Rg1 Ke8 Rh1 Kf7 Rg1 Ke8 Rh1 Kf7 Rg1 Ke8 Rh1 Kf7 Rg1 Ke8 Rh1 Kf7 Rg1 Ke8 Rh1 Kf7 Rg1 Ke8 Rh1 Kf7 Rg1 Ke8 Rh1 Kf7 Rg1 Ke8 Rh1 Kf7 Rg1 Ke8 Rh1 Kf7"
# history_list = chess_history.split(" ")

# for step in history_list:
#     # Convert SAN (human friendly) notation to a move object and UCI string
#     move = board.parse_san(step)
#     uci = move.uci()
#     move = Move.from_uci(uci)
#     board.push(move)  # Make the move
#     display(board)

In [ ]:
import re
from chess import Move, Board
import chess
from typing import List, Dict, Optional

def parse_game_file(file_path: str) -> List[Dict]:
    """
    Parse a text file containing chess games in the format:
    Game X: result (reason, move_count moves)
    move1 move2 move3 ...
    
    Returns a list of dictionaries, each containing:
    - game_number: int
    - result: str (e.g., "1/2-1/2")
    - reason: str (e.g., "MAX_MOVES")
    - move_count: int
    - moves: List[str] (SAN notation moves)
    """
    games = []
    
    with open(file_path, 'r') as f:
        content = f.read()
    
    # Pattern to match game headers: "Game X: result (reason, move_count moves)"
    game_pattern = r'Game (\d+):\s*([^\s]+)\s*\(([^,]+),\s*(\d+)\s*moves\)'
    
    # Split content by game headers
    parts = re.split(game_pattern, content)
    
    # Process each game (parts[0] is content before first game, then groups alternate)
    for i in range(1, len(parts), 5):
        if i + 4 < len(parts):
            game_number = int(parts[i])
            result = parts[i + 1]
            reason = parts[i + 2]
            move_count = int(parts[i + 3])
            moves_text = parts[i + 4].strip()
            
            # Parse moves (split by whitespace and filter empty strings)
            moves = [m for m in moves_text.split() if m]
            
            games.append({
                'game_number': game_number,
                'result': result,
                'reason': reason,
                'move_count': move_count,
                'moves': moves
            })
    
    return games

def replay_game(game_data: Dict, display_board: bool = True, step_delay: float = 0.0):
    """
    Replay a single game from parsed game data.
    
    Args:
        game_data: Dictionary containing game information (from parse_game_file)
        display_board: If True, display the board after each move
        step_delay: Delay in seconds between moves (for visualization)
    """
    board = Board()
    
    print(f"Game {game_data['game_number']}: {game_data['result']} ({game_data['reason']}, {game_data['move_count']} moves)")
    print(f"Total moves: {len(game_data['moves'])}")
    print("\nInitial position:")
    if display_board:
        display(board)
    
    for move_num, move_san in enumerate(game_data['moves'], 1):
        try:
            # Parse SAN notation to move object
            move = board.parse_san(move_san)
            board.push(move)
            
            if display_board:
                print(f"\nMove {move_num}: {move_san}")
                display(board)
            
            if step_delay > 0:
                import time
                time.sleep(step_delay)
                
        except Exception as e:
            print(f"Error at move {move_num} ({move_san}): {e}")
            break
    
    print(f"\nFinal position:")
    print(f"Outcome: {board.outcome()}")
    print(f"FEN: {board.fen()}")
    
    return board

def replay_all_games(file_path: str, game_numbers: Optional[List[int]] = None, 
                     display_board: bool = True, step_delay: float = 0.0):
    """
    Parse and replay games from a file.
    
    Args:
        file_path: Path to the game file
        game_numbers: Optional list of game numbers to replay (None = all games)
        display_board: If True, display the board after each move
        step_delay: Delay in seconds between moves
    """
    games = parse_game_file(file_path)
    
    if game_numbers is None:
        game_numbers = [g['game_number'] for g in games]
    
    for game in games:
        if game['game_number'] in game_numbers:
            replay_game(game, display_board=display_board, step_delay=step_delay)
            print("\n" + "="*80 + "\n")

# Example usage:
# games = parse_game_file("../games_iter_90.txt")
# replay_game(games[0], display_board=True)
# replay_all_games("../games_iter_90.txt", game_numbers=[1])